# Jetpack Compose - Paging3 - Podstawy

W tej aplikacji przyjrzymy się zastosowaniu biblioteki **Paging** w aplikacji.

<img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExd2UzaWtveDNyaXZ6aDhtaWs1bTJwcTByNGQ3ODRpbHdqbDF1emlyZiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/psiQ7oTjhymbdfqRVC/giphy.gif" width="200" />

Biblioteka `Paging3` to narzędzie do paginacji, które ułatwia ładowanie danych partiami w aplikacjach.

- Efektywne ładowanie danych: Biblioteka pomaga w efektywnym zarządzaniu ładowaniem dużych zbiorów danych, takich jak listy elementów w aplikacjach z wieloma rekordami. Dzięki niemu możesz ładować dane partiami, co pozwala na lepszą wydajność i oszczędność zasobów urządzenia.
- Aktualizacje w czasie rzeczywistym: Biblioteka umożliwia automatyczne monitorowanie źródła danych i dostarcza aktualizacje w czasie rzeczywistym, na przykład w odpowiedzi na zmiany w bazie danych lub na serwerze.
- Łatwe obsługiwanie błędów i odzyskiwanie: Biblioteka zawiera wbudowane mechanizmy obsługi błędów i odzyskiwania, co ułatwia zarządzanie sytuacjami takimi jak utrata połączenia internetowego lub błędy zapytań do źródła danych.
- Obsługa różnych źródeł danych: Możesz używać biblioteki z różnymi źródłami danych, takimi jak baza danych lokalna, zapytania sieciowe lub inne źródła danych.
- Integracja z interfejsem użytkownika: Biblioteka jest łatwa do zintegrowania z interfejsem użytkownika. Pozwala na płynne przewijanie i dynamiczne ładowanie nowych danych w miarę przewijania listy.
- Aby korzystać z biblioteki, musisz zdefiniować źródło danych `PagingSource`, które dostarcza dane `PagingData`. Biblioteka automatycznie zarządza ładowaniem danych w odpowiednich momentach, co ułatwia tworzenie wydajnych i responsywnych aplikacji Android.
- Biblioteka posiada wbudowane mechanizmy obsługi stanów (`LoadingState.Error`, `LoadingState.Append` etc.)

W tym przykładzie wykorzystamy [**SWAPI**](https://swapi.dev), który jest API posiadającym dane z uniwersum Star Wars. W tym przykładzie wykorzystamy endpoint *people* do wyświetlenia popularnych postaci.

Odpowiedź z serwera wygląda następująco:

In [ ]:
{
	"count": 82,
	"next": "https://swapi.dev/api/people/?page=2",
	"previous": null,
	"results": [
		{
			"name": "Luke Skywalker",
			"height": "172",
			"mass": "77",
			"hair_color": "blond",
			"skin_color": "fair",
			"eye_color": "blue",
			"birth_year": "19BBY",
			"gender": "male",
			"homeworld": "https://swapi.dev/api/planets/1/",
			"films": [
				"https://swapi.dev/api/films/1/",
				"https://swapi.dev/api/films/2/",
				"https://swapi.dev/api/films/3/",
				"https://swapi.dev/api/films/6/"
			],
			"species": [],
			"vehicles": [
				"https://swapi.dev/api/vehicles/14/",
				"https://swapi.dev/api/vehicles/30/"
			],
			"starships": [
				"https://swapi.dev/api/starships/12/",
				"https://swapi.dev/api/starships/22/"
			],
			"created": "2014-12-09T13:50:51.644000Z",
			"edited": "2014-12-20T21:17:56.891000Z",
			"url": "https://swapi.dev/api/people/1/"
		},
        ...
        ]
}

Bartdzo często w odpowiedzi znajdują się *metadane* zawierające różne informacje, tutaj mamy całkowitą liczbę rekordów (`count: 82`), linki do poprzedniej i następnej strony (`previous`, `next`), oraz dane (`result`). Kolejne porcje edanych otrzymujemy poprzez kojene zapytania `people/page`, każda strona zawiera 10 rekordów. W aplikacji będziemy ładować dane z tego endpointa po 10, dopóki nie załadujemy wszystkich 82 postaci. Nie będziemy obsługiwali stanów błędów i ładowania dla prostoty aplikacji i aby skupić się tylko na samej paginacji.

Nasza aplikacja wymaga dostępu do internetu, aby go uzyskać musimy dodać deklarację uprawnienia w pliku konfiguracyjnym aplikacji, który nazywa się `AndroidManifest.xml`. To uprawnienie informuje system operacyjny Android, że aplikacja potrzebuje dostępu do internetu.

In [ ]:
<uses-permission android:name="android.permission.INTERNET"/>

Następnie dodajmy wymagane zależności do projektu.

```kotlin
implementation ("androidx.paging:paging-runtime-ktx:3.2.0")

implementation ("com.squareup.retrofit2:retrofit:2.9.0")
implementation ("com.google.code.gson:gson:2.9.1")
implementation ("com.squareup.retrofit2:converter-gson:2.9.0")

// ViewModel
implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1")
//Fragment
implementation ("androidx.fragment:fragment-ktx:1.6.1")
```

Rozpocznijmy od utworzenia modelu danych odpowiadającego odpowiedzi z serwera. Tym razem wykorzystamy dwie klasy. `SwapiResponse` jest odpowiedzią z serwera zaweirającą nasze dane oraz kilka dodatkowych informacji o całkowitej liczbie rekordów, oraz linki do następnej i poprzedniej strony. Tutaj wykorzystamy typy *nullable*, ponieważ linki mogą nie istnieć (przykładowo, dla pierwszej strony nie istnieje *previous*).

Klasa `Result` reprezentuje jeden rekord danych, jak widzimy powyżej, postać jest scharakteryzowana prez wiele pól, nas będzie interesować tylko kilka, więc możemy zmapować wyłącznie pola które nas interesują - będziemy wykorzystywać tylko pole `name`, dla przykładu jest zapisywanych więcej informacji (`homeworld`, `height`)

In [ ]:
data class SwapiResponse(
    val count: Int,
    val next: String?,
    val previous: String?,
    val results: List<Result>
)

In [ ]:
data class Result(
    val height: String,
    @SerializedName("homeworld") val homeWorld: String,
    val name: String,
)

Przejdźmy do interfejsu reprezentującego api, zdefiniujemy jedną metodę, która jako parametr będzie przyjmowała numer strony.

In [ ]:
interface SwapiApiService {
    @GET("people")
    suspend fun getCharacters(
        @Query("page") page: Int
    ): SwapiResponse
}

Jak w poprzednich przykładach, dodajmy instancję `Retrofit`

In [ ]:
object RetrofitInstance {
    val api: SwapiApiService by lazy {
        Retrofit.Builder()
            .baseUrl("https://swapi.dev/api/")
            .addConverterFactory(GsonConverterFactory.create())
            .build()
            .create(SwapiApiService::class.java)
    }
}

Następnie zaimplementujmy repozytorium.

In [ ]:
class SwapiRepository {
    private val api = RetrofitInstance.api

    suspend fun getCharacters(page: Int) = api.getCharacters(page)
}

Aby wykorzystać paginację, musimy zdefiniować źródło danych `PagingSource`, dodajmy klasę `CharactersPagingSource`, rozszerzającą klasę `PagingSource`.

In [ ]:
class CharactersPagingSource(
    private val repository: SwapiRepository
) : PagingSource<Int, Result>() {
    override fun getRefreshKey(state: PagingState<Int, Result>): Int? {
        return state.anchorPosition ?.let { anchorPosition ->
            val anchorPage = state.closestPageToPosition(anchorPosition)
            anchorPage?.prevKey?.plus(1) ?: anchorPage?.nextKey?.minus(1)
        }
    }

    override suspend fun load(params: LoadParams<Int>): LoadResult<Int, Result> {
        return try {
            val page = params.key ?: 1
            val response = repository.getCharacters(page)

            LoadResult.Page(
                data = response.results,
                prevKey = getPageNumberFromUrl(response.previous),
                nextKey = getPageNumberFromUrl(response.next),
            )
        } catch (e: Exception) {
            LoadResult.Error(e)
        }
    }

    private fun getPageNumberFromUrl(url: String?): Int? {
        if (url != null) {
            val pattern = Pattern.compile("page=(\\d+)")
            val matcher = pattern.matcher(url)

            if (matcher.find()) {
                val pageNumberString = matcher.group(1)
                if (pageNumberString != null) {
                    return pageNumberString.toInt()
                }
            }
        }

        return null
    }
}

Omówmy dokładniej kod tej klasy. 

In [ ]:
class CharactersPagingSource(
    private val repository: SwapiRepository
) : PagingSource<Int, Result>() {

Ta klasa jest używana do źródła danych paginowanych dla listy postaci. `PagingSource<Int, Result>()`, jest specjalnym rodzajem źródła danych stworzonym do obsługi paginacji. `PagingSource` jest generycznym typem, który przyjmuje dwa argumenty generyczne: `Key` i `Value`.
- `Int` oznacza typ klucza, który jest używany do indeksowania stron danych. To zazwyczaj numer strony.
- `Result` oznacza typ danych, które są ładowane partiami. `Result` to typ reprezentujący postacie w aplikacji.

Ponieważ w odpowiedzi z serwera nie dostajemy bezpośrednio numeru strony, tylko cały link (`https://swapi.dev/api/people/?page=2`), dodajmy metodę pomocniczą, która nam ten numer zwróci na podstawie linku

In [ ]:
private fun getPageNumberFromUrl(url: String?): Int? {
    if (url != null) {
        val pattern = Pattern.compile("page=(\\d+)")
        val matcher = pattern.matcher(url)

        if (matcher.find()) {
            val pageNumberString = matcher.group(1)
            if (pageNumberString != null) {
                return pageNumberString.toInt()
            }
        }
    }

    return null
}

- `if (url != null) {`: Rozpoczynamy blok warunkowy, sprawdzając, czy przekazany URL nie jest `null`. Jeśli jest `null`, funkcja zwróci `null` jako wynik.
- `val pattern = Pattern.compile("page=(\\d+)")`: Tworzymy wyrażenie regularne za pomocą klasy `Pattern` w celu dopasowania do tekstu `"page="` oraz jednej lub więcej cyfr. To pozwoli nam na znalezienie fragmentu URL, który zawiera numer strony.
- `val matcher = pattern.matcher(url)`: Tworzymy obiekt `matcher` za pomocą wyrażenia regularnego `pattern` i przekazujemy mu analizowany URL.
- `if (matcher.find()) {`: Sprawdzamy, czy w tekście URL znaleziono dopasowanie do wzorca. Jeśli nie znaleziono, oznacza to, że URL nie zawiera informacji o numerze strony.
- `val pageNumberString = matcher.group(1)`: Jeśli znaleziono dopasowanie, używamy metody `group(1)` obiektu `matcher`, aby wyodrębnić pierwszą grupę dopasowania, która zawiera numer strony jako napis (`String`).
- `if (pageNumberString != null) {`: Sprawdzamy, czy numer strony nie jest `null` (może być `null`, jeśli nie znaleziono dopasowania lub nie udało się go wyodrębnić).
- `return pageNumberString.toInt()`: Jeśli numer strony jest dostępny i nie jest `null`, konwertujemy go na liczbę całkowitą za pomocą metody `toInt()` i zwracamy tę liczbę jako wynik funkcji.
- `return null`: Jeśli w którymś z etapów funkcji nie znaleziono numeru strony lub wystąpił błąd, funkcja zwraca `null`.

W klasie `PagingSource` musimy dostarczyć implementację dwóch metod:
- `refreshKey`:
    - `refreshKey` to metoda, która jest używana do określenia, jak należy odświeżyć źródło danych.
    - Funkcja ta jest wywoływana, gdy aplikacja chce odświeżyć całe źródło danych, na przykład gdy użytkownik przewinie listę w górę i chce załadować nowe dane na początku listy.
    - Wartość zwracana przez tę metodę (najczęściej jest to liczba lub inny identyfikator) określa klucz, który jest używany do identyfikacji pierwszej strony danych w źródle. Na podstawie tego klucza biblioteka `Paging` *wie*, od jakiego momentu rozpocząć ładowanie nowych danych.
- `load`:
    - `load` to metoda, która jest odpowiedzialna za ładowanie danych na podstawie klucza strony i rozmiaru strony.
    - Funkcja ta jest wywoływana, gdy użytkownik przewija listę i osiąga koniec dostępnych danych lub kiedy biblioteka `Paging` uznaje to za odpowiedni moment do załadowania kolejnej partii danych.
    - Metoda `load` przyjmuje dwa argumenty: `params: LoadParams<Key>` i `callback: LoadCallback<Key, Value>`.
        - `params` zawiera informacje, takie jak klucz strony, rozmiar strony i inne dane konfiguracyjne potrzebne do załadowania danych.
        - `callback` jest używany do przekazania załadowanych danych z powrotem do `Paging`, aby mogły być wyświetlone w interfejsie użytkownika.

Przejdźmy do metody `getRefreshKey`

In [ ]:
override fun getRefreshKey(state: PagingState<Int, Result>): Int? {
    return state.anchorPosition ?.let { anchorPosition ->
        val anchorPage = state.closestPageToPosition(anchorPosition)
        anchorPage?.prevKey?.plus(1) ?: anchorPage?.nextKey?.minus(1)
    }
}

- `state.anchorPosition`: To pozycja *kotwicy* (anchor position) w stanie paginacji (`PagingState`). Pozycja kotwicy to pozycja w widoku, który jest uważany za aktualnie wyświetlany przez użytkownika, np. pozycja pierwszego widocznego elementu na liście.
- `?.let { anchorPosition -> ... }`: Jest to blok, który wykonuje kod wewnątrz tylko wtedy, gdy `state.anchorPosition` nie jest `null`. Wartość `anchorPosition` jest dostępna wewnątrz tego bloku.
- `val anchorPage = state.closestPageToPosition(anchorPosition)`: Ta linia kodu pobiera stronę (`page`) najbliższą pozycji kotwicy. Strona w kontekście paginacji to fragment danych, który jest ładowany partiami.
- `anchorPage?.prevKey?.plus(1) ?: anchorPage?.nextKey?.minus(1)`: Ta linia kodu określa klucz odświeżania. Jeśli możliwy jest klucz poprzedniej strony (`prevKey`) dla strony kotwicy, to jest on inkrementowany o 1 (czyli następna strona). W przeciwnym razie, jeśli możliwy jest klucz następnej strony (`nextKey`) dla strony kotwicy, to jest on dekrementowany o 1 (czyli poprzednia strona). Jeśli oba klucze są `null`, wynikiem jest `null`.

Ogólnie rzecz biorąc, ta metoda stara się dostarczyć klucz odświeżania, który umożliwia paginację od punktu, który jest blisko aktualnie wyświetlanej pozycji. Klucz ten pozwala na efektywne odświeżenie danych na liście w odpowiednim miejscu, aby użytkownik mógł kontynuować przeglądanie bez większych przeskoków.

`getRefreshKey()` dostarcza klucz używany do początkowej ładowania dla następnego źródła paginacji ze względu na unieważnienie istniejącego źródła paginacji. Klucz ten jest dostarczany do załadowania za pomocą `LoadParams.key`. Ostatnia pozycja dostępu może być pobrana za pomocą `state.anchorPosition`.

Przykładowo:
Mając listę danych w działającej aplikacji, użytkownik przewija tę listę, a w międzyczasie dochodzi do zmiany w danych (aktualizacja/usunięcie itp.) Jeśli `getRefreshKey()` jest właściwie zaimplementowane, to biblioteka paginacji będzie *wiedziała*, jaka była ostatnia pozycja dostępu do danych. Wówczas lista nie *przeskoczy* na górę, a zamiast tego zostanie wyświetlona ostatnia przeglądana strona. To oznacza, że nawet po aktualizacji danych użytkownik pozostanie w tym samym miejscu w liście i nie musi przewijać listy od nowa.

Ostatnią metodą jest `load`. 

In [ ]:
override suspend fun load(params: LoadParams<Int>): LoadResult<Int, Result> {
    return try {
        val page = params.key ?: 1
        val response = repository.getCharacters(page)

        LoadResult.Page(
            data = response.results,
            prevKey = getPageNumberFromUrl(response.previous),
            nextKey = getPageNumberFromUrl(response.next),
        )
    } catch (e: Exception) {
        LoadResult.Error(e)
    }
}

- `override suspend fun load(params: LoadParams<Int>) : LoadResult<Int, Result>`: Ta metoda jest przesłoniętą funkcją z interfejsu `PagingSource` i jest używana do ładowania danych. Przyjmuje ona `params` jako argument, który zawiera klucz strony oraz rozmiar strony, który jest przekazywany przy żądaniu ładowania danych.
- `val page = params.key ?: 1`: Pobieramy klucz strony z `params`. Jeśli klucz jest `null` (co może się zdarzyć, na przykład podczas pierwszego ładowania), przyjmujemy domyślną wartość 1. To oznacza, że jeśli nie podano klucza strony, zostanie załadowana pierwsza strona danych.
- `val response = repository.getCharacters(page)`: Korzystając z repozytorium, wywołujemy funkcję `getCharacters(page)`, która pobiera dane postaci.
- `LoadResult.Page(...)`: Ta linia kodu tworzy obiekt `LoadResult.Page`, który jest używany do przekazania załadowanych danych z powrotem do biblioteki `Paging`. Wewnątrz tego obiektu przekazujemy:
- `data`: Dane postaci z odpowiedzi `response.results`, które są ładowane.
- `prevKey`: Wynik wywołania funkcji `getPageNumberFromUrl(response.previous)`. To jest poprzedni klucz strony, który jest używany w paginacji wstecznej.
- `nextKey`: Wynik wywołania funkcji `getPageNumberFromUrl(response.next)`. To jest następny klucz strony, który jest używany w paginacji do przodu.
- `catch (e: Exception) { LoadResult.Error(e) }`: To blok `try-catch`, który obsługuje ewentualne błędy podczas ładowania danych. Jeśli wystąpi błąd, zwracamy `LoadResult.Error`, a błąd jest przekazywany jako jego argument.

Ogólnie rzecz biorąc, ta metoda obsługuje ładowanie danych z określoną stroną i zwraca wynik w formie `LoadResult.Page` z danymi postaci oraz kluczami stron do paginacji wstecznej i do przodu. W przypadku błędu zwracany jest `LoadResult.Error` z informacją o błędzie.

Przejdźmy do `viewmodel`

In [ ]:
class SwapiViewModel : ViewModel() {
    private val repository = SwapiRepository()

    fun getData() = Pager(
        config = PagingConfig(
            pageSize = 10,
        ),
        pagingSourceFactory = {
            CharactersPagingSource(repository)
        }
    ).flow.cachedIn(viewModelScope)
}

- `fun getData() = Pager(...)`: Jest to funkcja, która zwraca strumień (`Flow`) paginowanych danych. Funkcja ta korzysta z biblioteki `Paging`, aby zarządzać paginacją danych.
- `Pager(...)` to konstruktor klasy `Pager`, który inicjuje mechanizm paginacji danych. Skonfigurowane są następujące parametry:
    - `config = PagingConfig(pageSize = 10)`: Konfiguracja paginacji zdefiniowana za pomocą `PagingConfig`. Określono, że rozmiar strony (`pageSize`) wynosi 10, co oznacza, że dane będą ładowane partiami po 10 elementów na stronę.
    - `pagingSourceFactory = { CharactersPagingSource(repository) }`: Właściwość `pagingSourceFactory` definiuje, jakie źródło danych (`CharactersPagingSource`) zostanie użyte do ładowania danych. Przekazano repozytorium jako argument do źródła danych.
- `.flow`: Po konfiguracji pagera, wywołujemy `.flow`, aby uzyskać strumień danych paginowanych, który może być obserwowany przez interfejs użytkownika.
- `.cachedIn(viewModelScope)`: Metoda `.cachedIn` zapewnia, że strumień danych będzie przechowywany w danym zakresie (`viewModelScope`), co jest ważne, aby uniknąć utraty danych w przypadku ponownego tworzenia `ViewModel` w trakcie cyklu życia aplikacji.

Ostatnim elementem będzie fragment oraz klasy wspierające `RecyclerView`. Rozpocznijmy od `RecyclerView`, klasy `ViewHolder` i `Comparator` pozostają takie jak w poprzednich przykładach.

In [ ]:
class CharacterViewHolder(private val binding: RvItemBinding) : RecyclerView.ViewHolder(binding.root) {
    fun bind(item: Result) {
            binding.name.text = item.name
    }
}

In [ ]:
class CharacterComparator : DiffUtil.ItemCallback<Result>() {
    override fun areItemsTheSame(oldItem: Result, newItem: Result): Boolean {
        return oldItem === newItem
    }

    override fun areContentsTheSame(oldItem: Result, newItem: Result): Boolean {
        return oldItem == newItem
    }
}

Różnicę zobaczymy w adapterze, `RecyclerView` posiada specjalny adapter do zarządzania paginacją danych `PagingAdaopter`

In [ ]:
class CharacterAdapter(characterComparator: CharacterComparator) : PagingDataAdapter<Result, CharacterViewHolder>(characterComparator) {
    override fun onBindViewHolder(holder: CharacterViewHolder, position: Int) {
        val item = getItem(position) ?: Result("", "", "")
        holder.bind(item)
    }

    override fun onCreateViewHolder(parent: ViewGroup, viewType: Int): CharacterViewHolder {
        return CharacterViewHolder(
            RvItemBinding.inflate(
                LayoutInflater.from(parent.context), parent, false
            )
        )
    }
}

Jak widzimy, sama implementacja nie różni się od, przykładowo, imple,entacji `Listadapter`, wszystkie metody do zarządzania paginacją są zaimplementowane w adapterze i nie musimy podejmować żadnych dodatkowych kroków.

In [ ]:
class ListFragment : Fragment() {

    private lateinit var binding: FragmentListBinding

    private val viewModel: SwapiViewModel by viewModels()

    override fun onCreateView(
        inflater: LayoutInflater, container: ViewGroup?,
        savedInstanceState: Bundle?
    ): View {
        binding = FragmentListBinding.inflate(inflater)

        val characterAdapter = CharacterAdapter(CharacterComparator())

        viewLifecycleOwner.lifecycleScope.launch {
            viewModel.getData().collectLatest { pagingData ->
                characterAdapter.submitData(pagingData)

            }
        }

        binding.recycler.apply {
            layoutManager = LinearLayoutManager(requireContext())
            adapter = characterAdapter
            setHasFixedSize(true)
        }


        return binding.root
    }
}

W samym fragmencie też nie ma zbytnich różnic, zamiast metody `submitList`, mamy metodę `submitData`, która przyjmuje `PagingData` jako argument.

Możemy przetestować aplikację.

<img src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExd2UzaWtveDNyaXZ6aDhtaWs1bTJwcTByNGQ3ODRpbHdqbDF1emlyZiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/psiQ7oTjhymbdfqRVC/giphy.gif" width="200" />